In [1]:
import json

In [2]:
rddElo = sc.textFile("hdfs://localhost:9000/user/emo/input/elo/*")
rddSumm = sc.textFile("hdfs://localhost:9000/user/emo/input/riot/*")

In [47]:
newRddElo = rddElo.map(lambda line: line.replace("\\", "\\\\")) \
                  .map(lambda line: json.loads(line, strict=False)).map(lambda line: (int(line['playerOrTeamId']), (line['tier'], line['rank'])))

newRddSumm = rddSumm.map(lambda line: line.replace("\\","\\\\")) \
                    .map(lambda line: json.loads(line,strict=False)).map(lambda line: (int(line['id']), (line['name'], line['summonerLevel'])))
    
riotJoinRdd = newRddSumm.join(newRddElo).distinct()
    
#riotJoinRdd.take(5)

In [15]:
s = '{ "_id" : { "$oid" : "5b536d335abc332c5d6a4ae1" }, "leagueId" : "be3620b0-0460-11e8-b952-f01fafdb579b", "leagueName" : "Morgana\'s Army", "tier" : "SILVER", "queueType" : "RANKED_SOLO_5x5", "rank" : "IV", "playerOrTeamId" : "43681224", "playerOrTeamName" : "D\ufffdX\x14\ufffd", "leaguePoints" : 28, "wins" : 27, "losses" : 36, "veteran" : false, "inactive" : false, "freshBlood" : false, "hotStreak" : false }'

In [38]:
newRddSumm.take(5)

[(23459460, ('Naapo', 28)),
 (34160621, ('Akatsukiii1', 30)),
 (87690392, ('Mayo k1ng', 23)),
 (3306772, ('Zugurt Aga', 30)),
 (82159365, ('x Gippy', 55))]

In [39]:
newRddElo.take(5)

[(43681224, ('SILVER', 'IV')),
 (82159365, ('SILVER', 'III')),
 (46174537, ('BRONZE', 'IV')),
 (43705880, ('GOLD', 'V')),
 (24675815, ('SILVER', 'II'))]

In [48]:
riotJoinRdd.take(10)

[(20765542, (('PistonKosiarz', 91), ('DIAMOND', 'I'))),
 (7704209, (('%l�|', 50), ('GOLD', 'IV'))),
 (36716171, (('BeLikeNick', 38), ('GOLD', 'III'))),
 (93918833, (('veggie shark ', 85), ('SILVER', 'II'))),
 (36575007, (('QuadtheGod', 123), ('GOLD', 'V'))),
 (29157634, (('Fresh X', 52), ('SILVER', 'V'))),
 (6530159, (('Miniduke', 66), ('GOLD', 'V'))),
 (94056888, (('Palulubu', 116), ('SILVER', 'II'))),
 (70600741, (('� �', 46), ('PLATINUM', 'IV'))),
 (20339393, (('DSaya', 105), ('DIAMOND', 'II')))]

In [49]:
riotJoinRdd.count()

404

In [58]:
riotRDD = riotJoinRdd.map(lambda line: (line[1][1][0], (line[1][0][1],1)))\
                     .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))\
                     .map(lambda line: (line[0], line[1][0]/line[1][1]))\
                     .saveAsTextFile("hdfs://localhost:9000/user/emo/output/AvgLvlPerTier")

In [54]:
riotRDD.collectAsMap()

{'BRONZE': 58.2,
 'CHALLENGER': 114.0,
 'DIAMOND': 89.70175438596492,
 'GOLD': 76.72380952380952,
 'MASTER': 100.57142857142857,
 'PLATINUM': 82.98630136986301,
 'SILVER': 70.62204724409449}

In [59]:
rddStreams = sc.textFile("hdfs://localhost:9000/user/emo/input/loltwitchstreams/*")
rddTwUsers = sc.textFile("hdfs://localhost:9000/user/emo/input/twitchusers/*")

In [85]:
def try_correct_json(json_string):
    try:
        t = json.loads(json_string,strict=False)
        return t
    except ValueError:
        try:
            try_to_correct_json = json_string + "}"
            t = json.loads(try_to_correct_json,strict=False)
            return t
        except ValueError:
            return "parseError"
        
        
newRddStreams = rddStreams.map(try_correct_json)\
                          .filter(lambda line: line != "parseError")\
                          .map(lambda line: (line['user_id'], line['viewer_count']))
        #map(lambda line: line.replace("\\","\\\\")) \
newRddTwUsers = rddTwUsers.map(try_correct_json)\
                          .filter(lambda line: line != "parseError")\
                          .map(lambda line: (line['id'], line['display_name']))

In [86]:
newRddStreams.take(5)

[('184804950', 1),
 ('28365930', 3),
 ('110350588', 5),
 ('40237278', 4),
 ('48130588', 2)]

In [87]:
newRddTwUsers.take(5)

[('23453248', 'DairySourCream'),
 ('152904586', 'brunablince'),
 ('115770266', 'nicebusinko'),
 ('152363392', 'x�ܤ'),
 ('60762465', 'LAVASPILLS')]

In [88]:
riotRDD2 = riotJoinRdd.map(lambda line: (line[1][0][0], line[1][1][0]))

twitchJoinRdd = newRddTwUsers.join(newRddStreams)\
                             .values()
                             
finalRDD = riotRDD2.join(twitchJoinRdd)\
                   .values()\
                   .map(lambda line: (line[0], (line[1],1)))\
                   .reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1]))\
                   .map(lambda line: (line[0], line[1][0]/line[1][1]))\
                   .collectAsMap()

In [89]:
finalRDD

{'BRONZE': 487.2608695652174,
 'CHALLENGER': 4022.625,
 'DIAMOND': 69.92,
 'GOLD': 374.38271604938274,
 'MASTER': 14.333333333333334,
 'PLATINUM': 291.53846153846155,
 'SILVER': 1606.2714285714285}